In [1]:
import image_similarity_measures
#from image_similarity_measures.quality_metrics import rmse, psnr
from image_similarity_measures.quality_metrics import metric_functions
from itertools import combinations
import cv2
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict
import time
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn_image as isns
from PIL import Image, ImageOps
import glob
#import PIL


In [2]:
#VAR declarations
all_metrics =  ['ssim']
# maximum umber of frames to capture
max_frames = 150

comparison_images = []

#comparison_images = [Image.open(img) for img in glob.glob("img/*.jpg")]
#image_labels = [img.split("/",1)[-1] for img in glob.glob("img/*.jpg")]
#comparison_images = [img for img in glob.glob("img/*.jpg")]

In [3]:
cap = cv2.VideoCapture('testvid.mov')
#Or most common ID for webcams:
#cap = cv2.VideoCapture(0)

In [4]:
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Frames per second using video.get(cv2.CAP_PROP_FPS) : {fps}")

Frames per second using video.get(cv2.CAP_PROP_FPS) : 30.029060381013885


In [5]:
frame_nr = 0
#ensure how to handle exeptions
while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        frame_name = f"frame_{frame_nr}.jpg" 
        print(frame_name)
        #cv2.imwrite(f"frame{frame_nr}.jpg", frame)
        #i.e. at 30 fps, this advances one second
        #int(fps) for rounding
        frame_nr += int(fps)  
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)
        comparison_images.append(tuple([frame_name,frame]))
    else:
        cap.release()
        break
frame_nr

frame_0.jpg
frame_30.jpg
frame_60.jpg
frame_90.jpg
frame_120.jpg
frame_150.jpg


180

In [6]:
comparison_images[0][0]

'frame_0.jpg'

In [7]:

#cap.release()

In [8]:
#def compare(df: pd.DataFrame, frame_a: Tuple[str, np.ndarray], frame_b:Tuple[str, np.ndarray], metrics: List[str]) -> pd.DataFrame:
def compare(df: pd.DataFrame, pair:Tuple[ Tuple[str, np.ndarray], Tuple[str, np.ndarray]], metrics: List[str]) -> pd.DataFrame:
#img_a = cv2.imread(path_a)
    #img_b = cv2.imread(path_b)
    img_name_a = pair[0][0]
    img_name_b = pair[1][0]
    
    img_a = pair[0][1]
    img_b = pair[1][1]
    

    for metric in metrics:
        metric_func = metric_functions[metric]
        start_time = time.time()
        out_value = float(metric_func(img_a, img_b))
        end_time = time.time()
        output_dict = {}
        output_dict['metric'] = metric
        output_dict['x'] = img_name_a
        output_dict['y'] = img_name_b
        output_dict['value'] = out_value
        output_dict['walltime'] = end_time - start_time
        df = df.append(output_dict,ignore_index=True)
        output_dict['x'] = img_name_a
        output_dict['y'] = img_name_b
        df = df.append(output_dict,ignore_index=True)
        
    return df


In [9]:
#df_00 = pd.DataFrame()
#for idx_x, x in enumerate(comparison_images):
#    for idx_y, y in enumerate(comparison_images):
        # remove duplicate comparisons: (x,y) == (y,x) or (x,x)
#        if idx_x >= idx_y:
#            continue
#        df_00 = compare(df_00, x, y, all_metrics)
#df


In [10]:
df_01 = pd.DataFrame()
for pair in combinations(comparison_images, 2):
    #print(pair)
    df_01 = compare(df_01, pair, all_metrics)
    #first, second = pair
    #print(f"first {first[0]}")
    #print(f"second {second[0]}")
    #print("-----")
    #if first[0] == second[0]:
        #continue
    #df_01 = compare(df_01, first, second, all_metrics)
df_01

,metric,x,y,value,walltime
0,ssim,frame_0.jpg,frame_30.jpg,0.946705,0.866579
1,ssim,frame_0.jpg,frame_30.jpg,0.946705,0.866579
2,ssim,frame_0.jpg,frame_60.jpg,0.913602,0.774975
3,ssim,frame_0.jpg,frame_60.jpg,0.913602,0.774975
4,ssim,frame_0.jpg,frame_90.jpg,0.902481,0.773100
5,ssim,frame_0.jpg,frame_90.jpg,0.902481,0.773100
6,ssim,frame_0.jpg,frame_120.jpg,0.918158,0.819410
7,ssim,frame_0.jpg,frame_120.jpg,0.918158,0.819410
8,ssim,frame_0.jpg,frame_150.jpg,0.914823,0.796734
9,ssim,frame_0.jpg,frame_150.jpg,0.914823,0.796734
